# More VectorStore QA

_**NOTE:** this uses Cassandra's experimental "Vector Similarity Search" capability.
At the moment, this is obtained by building and running an early alpha from a specific branch of the codebase._

In the previous Quickstart, you have created the inded and at the same time added the corpus of text to it.

In most cases, these two operations happen at different times: besides, often new documents keep being ingested.

This notebook demonstrates further interactions you can have with a Cassandra Vector Store.

It is assumed you have run the Quickstart notebook (so that the vector store is not empty)

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

The setup is similar to the one you saw:

In [2]:
from langchain.vectorstores.cassandra import Cassandra

In [3]:
from cqlsession import getLocalSession, getLocalKeyspace
localSession = getLocalSession()
localKeyspace = getLocalKeyspace()

In [4]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

llm = OpenAI(temperature=0)
myEmbedding = OpenAIEmbeddings()

**Note**: for the time being you have to explicitly _turn on this experimental flag_ on the `cassio` side:

In [5]:
import cassio
cassio.globals.enableExperimentalVectorSearch()

## Re-use an existing Vector Store

Creating this `Cassandra` vector store, it will re-connect with the existing data on DB.

In practice, you are loading an existing, pre-populated vector store for further usage.

_(make sure you are using the very same embedding function every time!)_

In [6]:
myCassandraVStore = Cassandra(
    embedding=myEmbedding,
    session=localSession,
    keyspace='demo',
    table_name='vs_test1',
)

These are some of the ways you can query the store:

In [7]:
myCassandraVStore.similarity_search_with_score(
    "Does anyone have a coughing fit?",
    k=1,
)

[(Document(page_content='"Nitre," I replied.  "How long have you had that cough?"\n\n"Ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh!\nugh! ugh!"\n\nMy poor friend found it impossible to reply for many minutes.\n\n"It is nothing," he said, at last.', metadata={'source': 'texts/amontillado.txt'}),
  0.905196785033554)]

### Adding new documents

Start with a very off-topic question, to demonstrate that no relevant documents are found (yet).

In [8]:
SPIDER_QUESTION = 'Compare Agelenidae and Lycosidae'
myCassandraVStore.similarity_search_with_relevance_scores(
    SPIDER_QUESTION,
    k=1,
    score_threshold=0.88,
)

/home/stefano/personal/Datastax/code/my_github/langchain/langchain/vectorstores/base.py:151: UserWarning: No relevant docs were retrieved using the relevance score                          threshold 0.88
  warnings.warn(


[]

You can add a couple of relevant paragraphs to the index, using the `add_texts` primitive:

In [9]:
spiderFacts = [
    """
    The Agelenidae are a large family of spiders in the suborder Araneomorphae.
    The body length of the smallest Agelenidae spiders are about 4 mm (0.16 in), excluding the legs,
    while the larger species grow to 20 mm (0.79 in) long. Some exceptionally large species,
    such as Eratigena atrica, may reach 5 to 10 cm (2.0 to 3.9 in) in total leg span.
    Agelenids have eight eyes in two horizontal rows of four. Their cephalothoraces narrow
    somewhat towards the front where the eyes are. Their abdomens are more or less oval, usually
    patterned with two rows of lines and spots. Some species have longitudinal lines on the dorsal
    surface of the cephalothorax, whereas other species do not; for example, the hobo spider does not,
    which assists in informally distinguishing it from similar-looking species.
    """,
    """
    Jumping spiders are a group of spiders that constitute the family Salticidae.
    As of 2019, this family contained over 600 described genera and over 6,000 described species,
    making it the largest family of spiders at 13% of all species.
    Jumping spiders have some of the best vision among arthropods and use it
    in courtship, hunting, and navigation.
    Although they normally move unobtrusively and fairly slowly,
    most species are capable of very agile jumps, notably when hunting,
    but sometimes in response to sudden threats or crossing long gaps.
    Both their book lungs and tracheal system are well-developed,
    and they use both systems (bimodal breathing).
    Jumping spiders are generally recognized by their eye pattern.
    All jumping spiders have four pairs of eyes, with the anterior median pair
    being particularly large. 
    """,
]
spiderMetadatas = [
    {'source': 'wikipedia/Agelenidae'},
    {'source': 'wikipedia/Salticidae'},
]
myCassandraVStore.add_texts(
    spiderFacts,
    spiderMetadatas,
)

['616746f0-fa57-11ed-b001-9f03b87aa84c',
 '616746f1-fa57-11ed-b001-9f03b87aa84c']

Another way is to add a text through LangChain's `Document` abstraction.

Note that, using one of LangChain's splitters, long input documents are made into (possibly overlapping) digestible chunks without much boilerplate:

In [10]:
mySplitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=120)

In [11]:
lycoText = """
Wolf spiders are members of the family Lycosidae.
They are robust and agile hunters with excellent eyesight.
They live mostly in solitude, hunt alone, and usually do not spin webs.
Some are opportunistic hunters, pouncing upon prey as they
find it or chasing it over short distances;
others wait for passing prey in or near the mouth of a burrow.

Wolf spiders resemble nursery web spiders (family Pisauridae),
but wolf spiders carry their egg sacs by attaching them to their spinnerets,
while the Pisauridae carry their egg sacs with their chelicerae and pedipalps.
Two of the wolf spider's eight eyes are large and prominent;
this distinguishes them from nursery web spiders,
whose eyes are all of roughly equal size.
This can also help distinguish them from the similar-looking grass spiders.
"""

lycoDocument = Document(
    page_content=lycoText,
    metadata={'source': 'wikipedia/Lycosidae'}
)

Use the splitter to "shred" the input document:

In [12]:
lycoDocs = mySplitter.transform_documents([lycoDocument])
lycoDocs

[Document(page_content='Wolf spiders are members of the family Lycosidae.\nThey are robust and agile hunters with excellent eyesight.\nThey live mostly in solitude, hunt alone, and usually do not spin webs.\nSome are opportunistic hunters, pouncing upon prey as they', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content='Some are opportunistic hunters, pouncing upon prey as they\nfind it or chasing it over short distances;\nothers wait for passing prey in or near the mouth of a burrow.', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content='Wolf spiders resemble nursery web spiders (family Pisauridae),\nbut wolf spiders carry their egg sacs by attaching them to their spinnerets,\nwhile the Pisauridae carry their egg sacs with their chelicerae and pedipalps.', metadata={'source': 'wikipedia/Lycosidae'}),
 Document(page_content="while the Pisauridae carry their egg sacs with their chelicerae and pedipalps.\nTwo of the wolf spider's eight eyes are large and p

These are ready to be added to the index:

In [13]:
myCassandraVStore.add_documents(lycoDocs)

['662bf3b6-fa57-11ed-b001-9f03b87aa84c',
 '662bf3b7-fa57-11ed-b001-9f03b87aa84c',
 '662bf3b8-fa57-11ed-b001-9f03b87aa84c',
 '662bf3b9-fa57-11ed-b001-9f03b87aa84c',
 '662bf3ba-fa57-11ed-b001-9f03b87aa84c']

### Querying the index (the store) again

Time to repeat the question:

In [14]:
myCassandraVStore.similarity_search_with_relevance_scores(
    SPIDER_QUESTION,
    k=3,
    score_threshold=0.88,
)

[(Document(page_content='\n    The Agelenidae are a large family of spiders in the suborder Araneomorphae.\n    The body length of the smallest Agelenidae spiders are about 4 mm (0.16 in), excluding the legs,\n    while the larger species grow to 20 mm (0.79 in) long. Some exceptionally large species,\n    such as Eratigena atrica, may reach 5 to 10 cm (2.0 to 3.9 in) in total leg span.\n    Agelenids have eight eyes in two horizontal rows of four. Their cephalothoraces narrow\n    somewhat towards the front where the eyes are. Their abdomens are more or less oval, usually\n    patterned with two rows of lines and spots. Some species have longitudinal lines on the dorsal\n    surface of the cephalothorax, whereas other species do not; for example, the hobo spider does not,\n    which assists in informally distinguishing it from similar-looking species.\n    ', metadata={'source': 'wikipedia/Agelenidae'}),
  0.902924684573849),
 (Document(page_content="while the Pisauridae carry their e

### Cleanup

You're done.

In order to leave the index empty for the next demo run, you may want to clean the index (i.e. empty the table on DB).

Just don't do that in production!

In [15]:
myCassandraVStore.delete_collection()